In [19]:
import os
import glob
import json
import pickle
from datetime import datetime
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

def process_articles(articles_metadata_path, articles_embeddings_path):
    """
    articles_metadata.csv와 articles_embeddings.pickle을 읽어, 
    각 기사에 대해 요약 텍스트와 embedding 정보를 포함하는 dict 생성.
    반환 형식:
      {
         article_id: {
             "summary": "title. category. author" (최대 200자),
             "embedding": [embedding vector as list of floats]
         },
         ...
      }
    """
    df = pd.read_csv(articles_metadata_path)
    
    """with open(articles_embeddings_path, "rb") as f:
        embeddings = pickle.load(f)
    if not isinstance(embeddings, np.ndarray):
        embeddings = np.array(embeddings)
    
    if df.shape[0] != embeddings.shape[0]:
        print("경고: articles_metadata.csv 행 수와 embedding 벡터 수가 일치하지 않습니다.")"""
    
    item_metadata = {}
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        article_id = int(row.get("article_id", ""))
        if not article_id:
            continue
        
        # title, category, author 등 유의미한 텍스트 정보 추출
        category_id = str(row.get("category_id", "")).strip()
        created_at_ts = datetime.fromtimestamp(int(row.get("created_at_ts", ""))/1000)
        publisher_id = str(row.get("publisher_id", "")).strip()
        words_count = str(row.get("words_count", "")).strip()
        
        summary_parts = []

        summary = f"Item ID: {article_id}, Category ID: {category_id}, Word count: {words_count} created at {created_at_ts}".strip()
        
        # embeddings의 idx번째 행을 리스트로 변환
        #embedding_vector = embeddings[idx].tolist() if idx < embeddings.shape[0] else []
        
        item_metadata[article_id] = summary
    return item_metadata

def safe_int(value, default=0):
    """numpy.int64, nan 등 기본형으로 변환"""
    try:
        if pd.isnull(value):
            return default
        return int(value)
    except Exception:
        try:
            return int(float(value))
        except Exception:
            return default

def process_clicks(clicks_folder):
    """
    clicks 폴더 내 모든 CSV 파일을 읽어, 사용자 인터랙션을 세션별로 그룹화합니다.
    CSV 컬럼:
      user_id, session_id, session_start, session_size,
      click_article_id, click_timestamp, click_environment,
      click_deviceGroup, click_os, click_country, click_region, click_referrer_type
    반환 형식:
      {
         user_id: [
             [  # 첫 번째 세션 (리스트 내 리스트)
                {
                  "article_id": ...,
                  "timestamp": ...,
                  "session_start": ...,
                  "session_size": ...,
                  "environment": ...,
                  "deviceGroup": ...,
                  "os": ...,
                  "country": ...,
                  "region": ...,
                  "referrer_type": ...
                },
                ...
             ],
             [  # 두 번째 세션
                ...
             ],
             ...
         ],
         ...
      }
    """
    clicks_pattern = os.path.join(clicks_folder, "*.csv")
    csv_files = glob.glob(clicks_pattern)
    
    user_sessions = {}  # {user_id: {session_id: [interaction, ...]}}
    
    for csv_file in tqdm(csv_files):
        df = pd.read_csv(csv_file)
        for _, row in df.iterrows():
            user_id = str(row.get("user_id", "")).strip()
            if not user_id:
                continue
            session_id = str(row.get("session_id", "")).strip() or "default"
            
            # 각 필드를 기본 파이썬 타입으로 변환
            interaction = {
                "item_id": str(row.get("click_article_id", "")).strip(),
                "timestamp": safe_int(row.get("click_timestamp", None), default=0),
                "additional_info":{
                    "session_start": bool(row.get("session_start", False)),
                    "session_size": safe_int(row.get("session_size", None), default=0),
                    "environment": str(row.get("click_environment", "")).strip(),
                    "deviceGroup": str(row.get("click_deviceGroup", "")).strip(),
                    "os": str(row.get("click_os", "")).strip(),
                    "country": str(row.get("click_country", "")).strip(),
                    "region": str(row.get("click_region", "")).strip(),
                    "referrer_type": str(row.get("click_referrer_type", "")).strip()
                }
            }
            
            if user_id not in user_sessions:
                user_sessions[user_id] = {}
            if session_id not in user_sessions[user_id]:
                user_sessions[user_id][session_id] = []
            user_sessions[user_id][session_id].append(interaction)
    
    # 세션별 정리: 각 사용자의 세션 딕셔너리를 리스트(세션의 리스트)로 변환
    user_interactions = {}
    for user, sessions in user_sessions.items():
        session_list = []
        for sess_id, interactions in sessions.items():
            interactions_sorted = sorted(
                interactions, 
                key=lambda x: x["timestamp"] if x["timestamp"] is not None else 0
            )
            session_list.append(interactions_sorted)
        session_list = sorted(
            session_list, 
            key=lambda sess: sess[0]["timestamp"] if (sess and sess[0]["timestamp"] is not None) else 0
        )
        user_interactions[user] = session_list
        
    # 사용자 아이디 기준 정렬된 dict로 변환 (sort_keys=True로 dump해도 되지만, 여기서 미리 정렬)
    user_interactions_sorted = {user: user_interactions[user] for user in sorted(user_interactions)}
    
    return user_interactions_sorted

articles_metadata_path = "articles_metadata.csv"
articles_embeddings_path = "articles_embeddings.pickle"
clicks_folder = os.path.join("clicks", "clicks")

item_metadata = process_articles(articles_metadata_path, articles_embeddings_path)
with open("item_metadata.json", "w", encoding="utf-8") as f:
    json.dump(item_metadata, f, ensure_ascii=False, indent=2, sort_keys=True)
print("Saved item_metadata.json")

"""user_interactions = process_clicks(clicks_folder)
with open("user_interactions.json", "w", encoding="utf-8") as f:
    json.dump(user_interactions, f, ensure_ascii=False, indent=2, sort_keys=True)
print("Saved user_interactions.json")"""



100%|██████████| 364047/364047 [00:07<00:00, 45877.63it/s]


Saved item_metadata.json


'user_interactions = process_clicks(clicks_folder)\nwith open("user_interactions.json", "w", encoding="utf-8") as f:\n    json.dump(user_interactions, f, ensure_ascii=False, indent=2, sort_keys=True)\nprint("Saved user_interactions.json")'

In [10]:
import json

# 파일 읽기
with open('interactions.json', 'r') as file:
    data = json.load(file)
test = False
# "data" 안의 각 interaction 순회
text_to_int = {}
current_int = 0
for interaction_key, interaction_value in data['data'].items():
    
    # 2차원 배열 순회
    for session in interaction_value:
        for interaction in session:
            # [2] 위치의 리스트 순회
            
            for i,info in enumerate(interaction[2]):
                if isinstance(info, int):
                    continue  # 이미 정수인 경우
                if isinstance(info, float):
                    continue  # 이미 정수인 경우
                elif isinstance(info, bool):
                    interaction[2][i] = 1 if info else 0  # bool을 1 또는 0으로 변환
                elif isinstance(info, str):
                    if info == "":
                        interaction[2][i] = -1  # 빈 문자열은 -1로 변환
                    else:
                        try:
                            interaction[2][i] = float(info)  # 실수로 변환 가능한 경우
                        except:
                            if info not in text_to_int:
                                text_to_int[info] = current_int
                                current_int += 1
                            interaction[2][i] = text_to_int[info]  # 텍스트를 정수로 변환
                else:
                    try:
                        interaction[2][i] = int(info)  # 정수로 변환 가능한 경우
                    except ValueError:
                        pass  # 변환 불가능한 경우는 그대로 둠

# 수정된 데이터를 저장 (필요한 경우)
with open('interaction_modified.json', 'w') as file:
    json.dump(data, file, indent=4)

In [22]:
import json
import numpy as np

def load_json(filename):
    with open(filename, "r", encoding="utf-8") as f:
        return json.load(f)

    
def compute_user_interactions_stats(user_interactions):
    total_users = len(user_interactions)
    total_sessions = 0
    total_interactions = 0
    session_counts = []  # 각 사용자의 세션 수
    interactions_per_session = []  # 각 세션당 interaction 수
    
    for user, sessions in user_interactions.items():
        num_sessions = len(sessions)
        session_counts.append(num_sessions)
        total_sessions += num_sessions
        for session in sessions:
            num_inter = len(session)
            interactions_per_session.append(num_inter)
            total_interactions += num_inter

    avg_sessions_per_user = np.mean(session_counts) if session_counts else 0
    avg_interactions_per_session = np.mean(interactions_per_session) if interactions_per_session else 0

    print("\n=== User Interactions Statistics ===")
    print(f"총 사용자 수: {total_users}")
    print(f"사용자당 평균 세션 수: {avg_sessions_per_user:.2f}")
    print(f"세션당 평균 interaction 수: {avg_interactions_per_session:.2f}")

def main():
    # JSON 파일 읽기
    item_metadata = load_json("item_metadata.json")
    user_interactions = load_json("user_interactions.json")
    
    # user_interactions는 user_id 기준 오름차순으로 저장되어 있다고 가정
    #compute_item_metadata_stats(item_metadata)
    compute_user_interactions_stats(user_interactions)

if __name__ == "__main__":
    main()



=== User Interactions Statistics ===
총 사용자 수: 322897
사용자당 평균 세션 수: 3.25
세션당 평균 interaction 수: 2.85


In [4]:
import pandas as pd
import json

# 파일 경로 (실제 경로에 맞게 수정)
events_path = os.path.join(directory_path, dataset["file_names"][1])
df_events = pd.read_csv(events_path)

# timestamp를 datetime으로 변환 (Retail Rocket의 timestamp는 밀리초초 단위라고 가정)
df_events['datetime'] = pd.to_datetime(df_events['timestamp'], unit='ms')
df_events = df_events.sort_values(by=['visitorid', 'datetime'])

# 필요한 정보만 추출: (itemid, timestamp, event, transactionid)
def extract_event(row):
    return {
        "itemid": row["itemid"],
        "timestamp": row["datetime"].strftime('%Y-%m-%d %H:%M:%S'),
        "event": row["event"],
        "transactionid": row["transactionid"] if pd.notna(row["transactionid"]) else ""
    }

df_events["event_info"] = df_events.apply(extract_event, axis=1)

# 사용자(visitorid)별로 그룹화하고, 시간 간격이 3시간(10800초) 이상이면 세션 분리
user_sessions = {}
three_hours = pd.Timedelta(seconds=10800)

for visitor, group in tqdm(df_events.groupby('visitorid')):
    group = group.sort_values('datetime')
    sessions = []
    current_session = []
    prev_time = None
    for _, row in group.iterrows():
        current_time = row['datetime']
        if prev_time is not None and (current_time - prev_time) >= three_hours:
            if current_session:
                sessions.append(current_session)
            current_session = []
        current_session.append(row["event_info"])
        prev_time = current_time
    if current_session:
        sessions.append(current_session)
    user_sessions[visitor] = sessions

# JSON 파일로 저장 (각 사용자별 세션을 포함)
with open("retailrocket_user_interactions.json", "w", encoding="utf-8") as f:
    json.dump(user_sessions, f, ensure_ascii=False, indent=2)

print("User interactions JSON이 'retailrocket_user_interactions.json' 파일로 저장되었습니다.")


100%|██████████| 1407580/1407580 [05:50<00:00, 4010.43it/s]


User interactions JSON이 'retailrocket_user_interactions.json' 파일로 저장되었습니다.


In [12]:
import json

# 기존 interaction JSON 파일 로드
with open("retailrocket_user_interactions.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

filtered_user_sessions = {}

for user, sessions in user_sessions.items():
    total_interactions = sum(len(session) for session in sessions)
    # 사용자별 세션 수가 3개 이상이고, 총 interaction 수가 5회 이상인 사용자만 선택
    if len(sessions) >= 2 and total_interactions >= 3:
        filtered_user_sessions[user] = sessions

# 필터링 결과를 새로운 JSON 파일로 저장
with open("retailrocket_user_interactions_filtered.json", "w", encoding="utf-8") as f:
    json.dump(filtered_user_sessions, f, ensure_ascii=False, indent=2)

print("Filtered interactions saved to 'retailrocket_user_interactions_filtered.json'.")


Filtered interactions saved to 'retailrocket_user_interactions_filtered.json'.


In [5]:
import json
import statistics

# JSON 파일 읽기
with open("retailrocket_user_interactions.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

# 사용자 수
num_users = len(user_sessions)

# 사용자별 세션 수와 전체 세션별 interaction 수 목록 계산
session_counts = []         # 각 사용자별 세션 수
interaction_counts = []     # 각 세션의 interaction 수

for user, sessions in tqdm(user_sessions.items()):
    session_counts.append(len(sessions))
    for session in sessions:
        interaction_counts.append(len(session))

# 사용자당 세션 수의 평균 계산
avg_sessions_per_user = sum(session_counts) / num_users if num_users > 0 else 0

# 각 세션당 interaction 수의 평균과 분산 계산
if interaction_counts:
    avg_interactions = statistics.mean(interaction_counts)
    var_interactions = statistics.variance(interaction_counts) if len(interaction_counts) > 1 else 0
else:
    avg_interactions = 0
    var_interactions = 0

print("Number of users:", num_users)
print("Average sessions per user:", avg_sessions_per_user)
print("Average interactions per session:", avg_interactions)
print("Variance of interactions per session:", var_interactions)


100%|██████████| 1407580/1407580 [00:00<00:00, 3024449.11it/s]


Number of users: 1407580
Average sessions per user: 1.196835703832109
Average interactions per session: 1.6360158419414925
Variance of interactions per session: 11.471367550718051


In [13]:
import json
import statistics

# JSON 파일 읽기
with open("retailrocket_user_interactions_filtered.json", "r", encoding="utf-8") as f:
    user_sessions = json.load(f)

# 사용자 수
num_users = len(user_sessions)

# 사용자별 세션 수와 전체 세션별 interaction 수 목록 계산
session_counts = []         # 각 사용자별 세션 수
interaction_counts = []     # 각 세션의 interaction 수

for user, sessions in tqdm(user_sessions.items()):
    session_counts.append(len(sessions))
    for session in sessions:
        interaction_counts.append(len(session))

# 사용자당 세션 수의 평균 계산
avg_sessions_per_user = sum(session_counts) / num_users if num_users > 0 else 0

# 각 세션당 interaction 수의 평균과 분산 계산
if interaction_counts:
    avg_interactions = statistics.mean(interaction_counts)
    var_interactions = statistics.variance(interaction_counts) if len(interaction_counts) > 1 else 0
else:
    avg_interactions = 0
    var_interactions = 0

print("Number of users:", num_users)
print("Average sessions per user:", avg_sessions_per_user)
print("Average interactions per session:", avg_interactions)
print("Variance of interactions per session:", var_interactions)


100%|██████████| 96057/96057 [00:00<00:00, 1001074.04it/s]


Number of users: 96057
Average sessions per user: 3.2782618653507813
Average interactions per session: 2.679044140997142
Variance of interactions per session: 49.42560094597455
